In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1TdfOz-MVrA_XP8h8oXd8IBTvOXwbFN1Y',
                                    dest_path='/content/data.zip',
                                    unzip=True)

Unzipping...

/usr/local/lib/python3.7/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1TdfOz-MVrA_XP8h8oXd8IBTvOXwbFN1Y" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [ ]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys


In [ ]:
print(torch.__version__) 
!git clone https://github.com/Zeeshanikramm/SignLanguageClassification.git
%cd SignLanguageClassification


1.10.0+cu111
Cloning into 'SignLanguageClassification'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
/content/SignLanguageClassification


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
asd=[1,2,3,4,5,6]

#Label file:
data_path ='/content/drive/MyDrive/signOutput'
classes = os.listdir(data_path)
print(len(classes))

for i in range(len(classes)):
  print(classes[i])
decoder = {}
for i in range(len(classes)):
    decoder[classes[i]] = i
encoder = {}
for i in range(len(classes)):
    encoder[i] = classes[i]

345
خ
د
ڈ
ذ
ر
ز
ڑ
ژ
س
ش
ض
ص
ط
ظ
ع
غ
ف
ق
ک
گ
م
ل
ن
ہ
ھ
و
ے
ء
ی
ا
آ
ب
پ
ت
ج
ث
ح
چ
کھانا
کاٹنا
کپڑے پہننا
کھرچنا
کھڑا ہونا
کہنا
کھینچنا
لینا
مدد کرنا
معاف کرنا
معذرت کرنا
ملاقات کا وقت لیجیے
نگلنا
ورزش کرنا
واضح کرنا
ہنسنا
وعدہ
اٹھنا
آرام
اشارہ کرنا
بات چیت
بچانا
باہر جانا
بھاگنا
بھولنا
پوچھنا
بولنا
بیٹھنا
پینا
خراش لگنا
خون بہنا
درست کرنا
دکھانا
دھکا دینا
دورہ کرنا
دیکھنا
دینا
ڈھونڈنا
روکنا
رابطہ کرنا
رکھنا
رونا
زخمی
سانس لینا
سننا
سونا
السلام وعلیکم
ادھر آو
آپ کیسے ہیں؟
ایک اچھا دن گزاریں
اچھا
ایمبولینس کو کال کریں
خبر دار
خدا حافظ
دروازہ بند کرو
دپہر کا سلام
شام بخیر
شب بخیر
مبارک ہو
عید مبارک
صبح بخیر
معاف کیجئے گا
میں آپ کی کیسے مدد کر سکتا ہوں؟
نیا سال مبارک ہو
چلے جاؤ
کبھی مجھ سے ملنے آو
چھونا مت
کھڑکی بند کرو
کوئی بات نہیں
کیا آپ اسکول جاتے ہیں؟
کیا آپ انگریزی بول سکتے ہیں؟
کیا آپ ایک سماعت کی مدد لیتے ہیں؟
کیا آپ تیار ہیں؟
کیا آپ کے پاس کوئی اسپرین ہے؟
کیا تم بالکل ٹھیک محسوس کرتے ہو؟
کیا تم بھوکے ہو؟
کیا تم بہرے ہو؟
کیا تم سمجھے؟
کیا تم نے اس سے پوچھا؟
کیا تم نے ک

In [ ]:
id = list()
path = '/content/drive/MyDrive/signOutput'
for i in os.listdir(path):
  p1 = os.path.join(path,i)
  for j in os.listdir(p1):
    p2 = os.path.join(p1,j)
    id.append((i,p2))

In [ ]:
class video_dataset(Dataset):
    def __init__(self,frame_list,sequence_length = 16,transform = None):
        self.frame_list = frame_list
        self.transform = transform
        self.sequence_length = sequence_length
    def __len__(self):
        return len(self.frame_list)
    def __getitem__(self,idx):
        label,path = self.frame_list[idx]
        # print("Path is ",path)
        # print("label is ", label)
        list_of_imgs_dir=os.listdir(path)

        
        count=0
        seq_img = list()
        for i in range(12):   # CHANGING for i in range(16): to for i in range(6): -----> now 3
            #print(list_of_imgs_dir[0:8], "   The 0th is " , list_of_imgs_dir[0] )
            try:
              count=count+1
              img = cv2.imread(path+"/"+list_of_imgs_dir[0])
              # print("--------")
              list_of_imgs_dir.pop(0)
              img1 = img[:,:,:]
              #cv2.imshow("img1",img1)
              if(self.transform):
                img1 = self.transform(Image.fromarray(img))
              seq_img.append(img1)
            except:
              break
        seq_image = torch.stack(seq_img)
        seq_image = seq_image.reshape(3,12,im_size,im_size) # chaning seq_image = seq_image.reshape(3,16,im_size,im_size) TO seq_image = seq_image.reshape(3,6,im_size,im_size) ---> now 3
        count=0
        return seq_image,decoder[label]

In [ ]:
im_size = 128
mean = [0.4889, 0.4887, 0.4891]
std = [0.2074, 0.2074, 0.2074]


train_transforms= transforms.Compose([
        transforms.Resize(size=(128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

train_data = video_dataset(id,sequence_length = 16,transform = train_transforms)
train_loader = DataLoader(train_data,batch_size = 32,num_workers = 4 ,shuffle = True)    
dataloaders = {'train':train_loader}


#train_transforms = transforms.Compose([
#                                        transforms.Resize((im_size,im_size)),
#                                        transforms.RandomHorizontalFlip(),
#                                        transforms.RandomRotation(degrees=10),
#                                        transforms.ToTensor(),
#                                        transforms.Normalize(mean,std)])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from model import resnet50
model = resnet50(class_num=345).to('cuda')  

In [ ]:
from clr import *
device = 'cuda'
cls_criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum = 0.9,weight_decay = 1e-4)
num_epochs = 60
onecyc = OneCycle(len(train_loader)*num_epochs,1e-3)

In [ ]:
from PIL import Image

In [ ]:
from collections import OrderedDict

#This code is to append Loss and Accuray to plot them lateer--------------------------
def append_new_line(file_name, text_to_append):
    """Append given text as a new line at the end of file"""
    # Open the file in append & read mode ('a+')
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)







#os.makedirs('/content/drive/MyDrive/sign_Weights2',exist_ok = True)
if not (os.path.exists('/content/drive/MyDrive/sign_Weights5')):
    os.makedirs('/content/drive/MyDrive/sign_Weights5',exist_ok = True)
from torch.autograd import Variable
iteration = 0
acc_all = list()
loss_all = list()


#model.load_state_dict(torch.load(PATH))
    
check =0

for epoch in range(num_epochs):
    if (os.path.exists('/content/drive/MyDrive/sign_Weights5/c3d_{}.h5'.format(epoch))):
        print(" epoch" , epoch , "is present")
        check=1
        continue
    #torch.load('/content/drive/MyDrive/sign_Weights2/c3d_{}.h5'.format(epoch))
    if (check==1):
        print("Loading saved model")
        #model =  resnet50(class_num=345).to('cuda') 
        #model.load_state_dict(torch.load('/content/drive/MyDrive/sign_Weights2/c3d_{}.h5'.format(epoch-1) ))
        #model.eval()


        model = resnet50(class_num=345).to('cuda') 
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum = 0.9,weight_decay = 1e-4)
        checkpoint = torch.load('/content/drive/MyDrive/sign_Weights5/c3d_{}.h5'.format(epoch-1) )
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        loss = checkpoint['loss']

        model.train()
        check=0
# - or -
        #model.train()
       
    #torch.eval()
    #model = TempModel()
#model.load_state_dict(torch.load(file_path))
#model.eval()
    print('')
    print(f"--- Epoch {epoch} ---")
    phase1 = dataloaders.keys()
    for phase in phase1:
       # torch.load  
        print('')
        print(f"--- Phase {phase} ---")
        epoch_metrics = {"loss": [], "acc": []}
        
        for batch_i, (X, y) in enumerate(dataloaders[phase]):
            #iteration = iteration+1
            image_sequences = Variable(X.to(device), requires_grad=True)
            labels = Variable(y.to(device), requires_grad=False)
            optimizer.zero_grad()
            #model.lstm.reset_hidden_state()
            predictions = model(image_sequences)
            loss = cls_criterion(predictions, labels)
            acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
            loss.backward()
            optimizer.step()
            epoch_metrics["loss"].append(loss.item())
            epoch_metrics["acc"].append(acc)
            if(phase=='train'):
                lr,mom = onecyc.calc()
                update_lr(optimizer, lr)
                update_mom(optimizer, mom)
            batches_done = epoch * len(dataloaders[phase]) + batch_i
            batches_left = num_epochs * len(dataloaders[phase]) - batches_done
            sys.stdout.write(
                    "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
                    % (
                        epoch,
                        num_epochs,
                        batch_i,
                        len(dataloaders[phase]),
                        loss.item(),
                        np.mean(epoch_metrics["loss"]),
                        acc,
                        np.mean(epoch_metrics["acc"]),
                    )
                )

                # Empty cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
        print('')
        print('{} , acc: {}'.format(phase,np.mean(epoch_metrics["acc"])))
        #torch.save(model.state_dict(),'/content/drive/MyDrive/sign_Weights2/c3d_{}.h5'.format(epoch))

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss}, '/content/drive/MyDrive/sign_Weights5/c3d_{}.h5'.format(epoch))



        if(phase=='train'):
          
          acc_all.append(np.mean(epoch_metrics["acc"]))
          loss_all.append(np.mean(epoch_metrics["loss"]))
          append_new_line('/content/drive/MyDrive/sign_Weights5/losslist.txt',str(np.mean(epoch_metrics["loss"])))
          append_new_line('/content/drive/MyDrive/sign_Weights5/acclist.txt',str(np.mean(epoch_metrics["acc"])))

 epoch 0 is present
 epoch 1 is present
 epoch 2 is present
 epoch 3 is present
 epoch 4 is present
 epoch 5 is present
 epoch 6 is present
 epoch 7 is present
 epoch 8 is present
 epoch 9 is present
 epoch 10 is present
 epoch 11 is present
 epoch 12 is present
 epoch 13 is present
 epoch 14 is present
 epoch 15 is present
 epoch 16 is present
 epoch 17 is present
 epoch 18 is present
 epoch 19 is present
 epoch 20 is present
 epoch 21 is present
 epoch 22 is present
 epoch 23 is present
 epoch 24 is present
 epoch 25 is present
 epoch 26 is present
 epoch 27 is present
 epoch 28 is present
 epoch 29 is present
 epoch 30 is present
 epoch 31 is present
 epoch 32 is present
 epoch 33 is present
 epoch 34 is present
 epoch 35 is present
 epoch 36 is present
 epoch 37 is present
 epoch 38 is present
 epoch 39 is present
 epoch 40 is present
 epoch 41 is present
 epoch 42 is present
 epoch 43 is present
 epoch 44 is present
 epoch 45 is present
 epoch 46 is present
 epoch 47 is present
 e

In [ ]:
loss_all

[1.5509752490941215,
 1.5066994937027203,
 1.5412968765286839,
 1.4979738010203136,
 1.5038245381677853,
 1.4676428901798584,
 1.406291732455001,
 1.4469075733247925]

In [ ]:

#============DONT UNCOMMENT==================

'''def append_new_line(file_name, text_to_append):
    """Append given text as a new line at the end of file"""
    # Open the file in append & read mode ('a+')
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)

'''
  


txt_file = open("/content/drive/MyDrive/sign_Weights4/losslist.txt", "r")
file_content = txt_file.read()
#print("The file content are: ", file_content)

content_list = file_content.split("\n")
txt_file.close()
#print("The list is: ", content_list)
accuracy_array=list()
for i in content_list:
    accuracy_array.append(float(i))
print("Accuracy list is: ", accuracy_array)


txt_file = open("/content/drive/MyDrive/sign_Weights4/acclist.txt", "r")
file_content = txt_file.read()
#print("The file content are: ", file_content)

content_list = file_content.split("\n")
txt_file.close()
#print("The list is: ", content_list)
loss_array=list()
for i in content_list:
    loss_array.append(float(i))
print("Loss list is: ", loss_array)

FileNotFoundError: ignored

In [ ]:
def error_plot(loss):
    plt.figure(figsize=(10,5))
    plt.plot(loss)
    plt.title("Training loss plot")
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.show()
def acc_plot(acc):
    plt.figure(figsize=(10,5))
    plt.plot(acc)
    plt.title("Training accuracy plot")
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.show()

In [ ]:

acc_plot(accuracy_array)

In [ ]:
error_plot(loss_array)


In [ ]:
from collections import OrderedDict
id = list()
count =0
path = '/content/drive/MyDrive/excuse'
for i in os.listdir(path):
  p1 = os.path.join(path,i)
  for j in os.listdir(p1):
    p2 = os.path.join(p1,j)
    id.append((i,p2))

In [ ]:
from clr import *
device = 'cuda'
from torch.autograd import Variable

In [ ]:
im_size = 128
mean = [0.4889, 0.4887, 0.4891]
std = [0.2074, 0.2074, 0.2074]


test_transforms= transforms.Compose([
        transforms.Resize(size=(128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])



test_data = video_dataset(id,sequence_length = 16,transform = test_transforms)
test_loader = DataLoader(test_data,batch_size = 64,num_workers = 4 ,shuffle = True)    
testdataloaders = {'test':test_loader}






#test_loader = DataLoader(test_data,batch_size = 32,num_workers = 4 ,shuffle = True)    
#dataloaders = {'train':test_loader}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
model = resnet50(class_num=345).to('cuda') 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum = 0.9,weight_decay = 1e-4)
epoch = 60
for i in range (55,epoch):
  try:
    checkpoint = torch.load('/content/drive/MyDrive/sign_Weights5/c3d_{}.h5'.format(i))
  except:
    continue
  #checkpoint = torch.load('/content/drive/MyDrive/sign_Weights4/c3d_30.h5')

  model.load_state_dict(checkpoint['model_state_dict'])
  model.eval()


  lab=[]
  pre=[]

  phase1 = testdataloaders.keys()
  for phase in phase1:
      # torch.load  
      print('')
      print(f"--- Phase {phase} ---")
      epoch_metrics = {"loss": [], "acc": []}
      
      for batch_i, (X, y) in enumerate(testdataloaders[phase]):
          #iteration = iteration+1
          image_sequences = Variable(X.to(device), requires_grad=True)
          labels = Variable(y.to(device), requires_grad=False)
          
          #model.lstm.reset_hidden_state()
          predictions = model(image_sequences)
          print(predictions.shape)

          loss = cls_criterion(predictions, labels)
          #pre.append(predictions.detach().argmax(1))
          #lab.append(lab)
          acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()



        




          epoch_metrics["loss"].append(loss.item())
          epoch_metrics["acc"].append(acc)
              # Empty cache
          
          
      print('')
      print('{} , acc: {}'.format(phase,np.mean(epoch_metrics["acc"])) , " on epoch " , i)
      #torch.save(model.state_dict(),'/content/drive/MyDrive/sign_Weights2/c3d_{}.h5'.format(epoch))




--- Phase test ---


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([5, 345])

test , acc: 80.0  on epoch  55

--- Phase test ---
torch.Size([5, 345])

test , acc: 60.0  on epoch  56

--- Phase test ---
torch.Size([5, 345])

test , acc: 80.0  on epoch  57

--- Phase test ---
torch.Size([5, 345])

test , acc: 60.0  on epoch  58

--- Phase test ---
torch.Size([5, 345])

test , acc: 40.0  on epoch  59


In [ ]:
print(pre)

In [ ]:
predictions

In [ ]:
predictions.detach().argmax(1)

In [ ]:
labels